In [ ]:
import asyncio
import io
import aiohttp
from PIL import Image

image_urls = [
    # Credit: Scott Foster
    "https://neont.s3.amazonaws.com/wp-content/uploads/2017/11/Scott-Foster-Too-Pooped-to-Party.jpg",
    # Credit: Katie McPherson
    "https://storageciggallery.addons.business/13611/cig-cozy-gallery-6892vUf-Katie-McPherson-CA-hd.jpg?c=00",
    # Credit: Kyle Finger
    "https://storageciggallery.addons.business/13611/cig-cozy-gallery-6892nr8-KyleFingerWisconsin-hd.jpg?c=00",
    # Credit: Cayuga Nature Center
    "https://images.squarespace-cdn.com/content/v1/5d3cb13b96f9ac0001e89cf6/1578338468978-4U1LVRPRLU2B83UR77BX/Smith-Woods-trees.JPG",
]


async def fetch_image(session: aiohttp.ClientSession, url: str) -> Image.Image:
    async with session.get(url) as response:
        image_bytes = await response.read()
        return Image.open(io.BytesIO(image_bytes))


async def fetch_images(urls: list[str]) -> list[Image.Image]:
    async with aiohttp.ClientSession() as session:
        return await asyncio.gather(*[fetch_image(session, url) for url in urls])


def get_bytes_from_image(image: Image.Image) -> bytes:
    img_byte_arr = io.BytesIO()
    image.save(img_byte_arr, format="JPEG")
    return img_byte_arr.getvalue()


images = await fetch_images(image_urls)

In [ ]:
from IPython.display import display  # pyright: ignore

for image in images:
    display(image)

In [ ]:
from typing import Any

MODEL_NAME = "dragoneye/animals"
# Fill this in with your auth token
AUTH_TOKEN = "<YOUR_AUTH_TOKEN>"


async def get_prediction(
    session: aiohttp.ClientSession,
    model_name: str,
    image: Image.Image,
) -> dict[str, Any]:
    async with session.post(
        "https://api.dragoneye.ai/predict",
        data={
            "image_file": get_bytes_from_image(image),
            "model_name": model_name,
        },
        headers={
            "Authorization": f"Bearer {AUTH_TOKEN}",
        },
    ) as response:
        return await response.json()


async with aiohttp.ClientSession() as session:
    prediction_results = await asyncio.gather(*[
        get_prediction(session, MODEL_NAME, image) for image in images
    ])
    images_with_prediction_results = zip(images, prediction_results)

In [ ]:
def all_objects_in_image(
    prediction_results: dict[str, Any],
    prediction_threshold: float = 0.7,
) -> list[tuple[str, float]]:
    def get_category(prediction: dict[str, Any]) -> tuple[str, float]:
        """
        Parse out the category from the hierarchical prediction.
        """
        if not prediction["children"]:
            return (prediction["name"], prediction["score"])
        else:
            return get_category(
                prediction["children"][0],
            )

    predictions = [
        get_category(prediction["category"])
        for prediction in prediction_results["predictions"]
    ]

    return [
        prediction for prediction in predictions if prediction[1] > prediction_threshold
    ]


images_with_objects_in_image = [
    (image, all_objects_in_image(prediction_results))
    for image, prediction_results in images_with_prediction_results
]


In [ ]:
for image, objects_in_image in images_with_objects_in_image:
    print(f"Animals detected in image: {set([name for name, _ in objects_in_image])}")
    display(image)